In [0]:
import pandas as pd
import numpy as np
import torch
from torch.utils import data
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder
from torch.utils.data.sampler import SubsetRandomSampler


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
class ECG_dataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.ecg_frame = pd.read_csv(csv_file, index_col = 0)
#        self.ecg_frame = self.ecg_frame.drop(columns = ['Unnamed: 0'])
        self.transform = transform
        le = LabelEncoder()
        self.ecg_frame.Label = le.fit_transform(self.ecg_frame.Label)

    def __len__(self):
        return len(self.ecg_frame)
    
    def __getitem__(self, idx):
      if torch.is_tensor(idx):
        idx = idx.tolist()
      signal = np.asarray(self.ecg_frame.iloc[idx, :-1].values).reshape(1,1000).astype('double')
      signal = torch.from_numpy(signal).type(torch.FloatTensor)
      label = self.ecg_frame.Label.values[idx]    
      label = torch.from_numpy(label).type(torch.LongTensor)

      #sample = {'signal': signal, 'label': label}
      if self.transform:
          signal_as_tensor = self.transform(signal)[0][0]
      return (signal, label)

In [0]:
csv = '/content/gdrive/My Drive/ECG_data/test_1000.csv'
transformations = transforms.Compose([transforms.ToTensor()])

ECG_data = ECG_dataset(csv_file = csv)
dataloader = DataLoader(ECG_data, batch_size=4,
                        shuffle=True, num_workers=4)

In [5]:
for _, (x, y) in enumerate(dataloader):
    print(x,y)
    break

tensor([[-0.0225, -0.0106,  0.0262,  ...,  0.0183,  0.0267,  0.0286],
        [-0.0623, -0.1057, -0.0854,  ..., -0.4133, -0.4102, -0.4282],
        [-0.1081, -0.1729, -0.1528,  ..., -0.1379, -0.1565, -0.1418],
        [-0.4066, -0.4324, -0.4701,  ..., -0.0554, -0.1259, -0.2175]]) tensor([0, 0, 0, 0])


In [17]:
# Train test split
validation_split = .2
shuffle_dataset = True
random_seed= 42
dataset_size = len(ECG_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(ECG_data, batch_size=128, 
                                           sampler=train_sampler)
validation_loader = DataLoader(ECG_data, batch_size=128,
                                                sampler=valid_sampler)
for _, (x,y) in enumerate(train_loader):
  print(x,y)
  break

#net, loss, val_score = learn(train_loader, train_loader)


tensor([[-0.5118, -0.5272, -0.4968,  ..., -0.4109, -0.4119, -0.4249],
        [ 0.0850, -0.1758, -0.2695,  ...,  0.0355,  0.0327,  0.0501],
        [ 0.0617,  0.0758,  0.0551,  ...,  0.1438,  0.1529,  0.1456],
        ...,
        [-0.1287, -0.1460, -0.1423,  ...,  0.0092,  0.0221,  0.0009],
        [-0.3484, -0.0227,  0.5873,  ..., -0.4178, -0.4016, -0.4140],
        [-0.1950, -0.1750, -0.1950,  ...,  0.2450,  0.3250,  0.3350]]) tensor([0, 0, 0, 2, 1, 0, 0, 2, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
        0, 0, 2, 0, 0, 1, 0, 0, 1, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 1,
        0, 0, 2, 1, 0, 0, 1, 0, 0, 2, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 2,
        1, 0, 0, 0, 0, 1, 2, 0, 1, 0, 2, 0, 2, 1, 0, 2, 1, 0, 2, 0, 0, 0, 2, 1,
        0, 0, 1, 2, 2, 0, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1,
        2, 2, 0, 0, 0, 0, 0, 2])
